In [1]:
!pip install tensorflow kaggle keras scikit-learn visualkeras keras.utils fastai torchvision keras4torch

In [2]:
import os
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from keras.preprocessing.image import ImageDataGenerator
import keras
import time
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Input
from keras.layers import Reshape
from keras.models import Model
from keras.layers import Conv1D, Conv2D, Conv3D, SeparableConv1D, MaxPooling1D, Activation, MaxPooling2D, Dense, Flatten, GlobalAveragePooling2D, BatchNormalization, Conv2D, Dropout
from tensorflow.keras.applications.imagenet_utils import preprocess_input
from tensorflow.keras import layers
from keras.applications import VGG19
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.initializers import RandomNormal
from fastai.basics import untar_data, URLs, Mish, MaxPool, LabelSmoothingCrossEntropy, ranger
import torchvision
import keras4torch as k4t
from torchvision import datasets, models, transforms
from fastai.vision.models.xresnet import *
import resource

2023-10-01 16:51:16.355981: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
resource.setrlimit(resource.RLIMIT_DATA, (42949672960, 42949672960))

In [4]:
def rgb2gray(images):
    return np.expand_dims(np.dot(images, [0.2990, 0.5870, 0.1140]), axis=3)

In [5]:
def lr_scheduler(epoch, lr):
    if epoch % 10 == 0 and epoch != 0:
        lr = lr / 2
    return lr

In [6]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix', cmap=plt.cm.tab20b):
    """
    This function prints and plots the confusion matrix. 
    Normalization can be applied by setting normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm. sum (axis=1)[:, np.newaxis] 
        print("Normalized confusion matrix")
    else:    
        print('Confusion matrix, without normalization')
    print(cm)
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product (range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
            horizontalalignment="center",
            color="white" if cm[i, j]> thresh else "black")
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [7]:
def create_keras_model():
    
    keras.backend.clear_session()

    tf.keras.backend.set_image_data_format("channels_first")
    
    initializer = RandomNormal(mean=0.0, stddev=0.01)

    initializer2 = RandomNormal(mean=0.0, stddev=0.01)
    
    model = Sequential()
    
    model.add(keras.applications.resnet.ResNet50(include_top=False,input_shape=(3,128,128), pooling='avg', weights='imagenet'))

    model.add(Flatten())
    
    model.add(BatchNormalization())
    
    model.add(Dense(2048, activation='relu', kernel_initializer=initializer))
    
    model.add(Dropout(0.3))

    model.add(BatchNormalization())
    
    model.add(Dense(1024, activation='relu', kernel_initializer=initializer2))
    
    model.add(Dropout(0.4))
    
    model.add(Dense(10, activation='softmax'))

    model.layers[0].trainable = False
    
    optimizer = keras.optimizers.Adam(amsgrad=True, clipnorm=1.0, learning_rate=1e-2)
    
    model.compile(optimizer=optimizer,
    loss='categorical_focal_crossentropy',
    metrics=['accuracy'])
    
    return model

In [8]:
path = untar_data(URLs.IMAGENETTE_320)
traindir = os.path.join(path, 'train')
valdir = os.path.join(path, 'val')

In [9]:
train_dataset = datasets.ImageFolder(
    traindir,
    transforms.Compose([
        transforms.Resize(128),
        transforms.CenterCrop(128),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ])
)

val_dataset = datasets.ImageFolder(
    valdir,
    transforms.Compose([
        transforms.Resize(128),
        transforms.CenterCrop(128),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ])
)

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=128, shuffle=False,
    num_workers=1, pin_memory=False) 

val_loader = torch.utils.data.DataLoader(
    val_dataset, batch_size=127, shuffle=False,
    num_workers=1, pin_memory=False)

In [10]:
X_train = []
y_train = []
for batch in train_loader:
    images, labels = batch
    X_train.append(images)
    y_train.append(labels)
print(X_train.shape)

ERROR: Unexpected bus error encountered in worker. This might be caused by insufficient shared memory (shm).
 

RuntimeError: DataLoader worker (pid(s) 24460) exited unexpectedly

In [ ]:
class_names = train_dataset.classes

In [ ]:
plt.figure(figsize=(10, 10))
sample_images, sample_labels = next(iter(train_loader))
sample_images = sample_images.numpy()
sample_labels = sample_labels.numpy()
for idx in range(9):
    ax = plt.subplot(3, 3, idx + 1)
    image = sample_images[idx].transpose((1, 2, 0))
    plt.imshow(image)
    plt.title("Ground Truth Label: " + class_names[int(sample_labels[idx])])
    plt.axis("off")

In [ ]:
datagen = ImageDataGenerator(rotation_range=8,
                             zoom_range=[0.95, 1.05],
                             height_shift_range=0.15,
                             shear_range=0.2)

In [ ]:
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

model_checkpoint = keras.callbacks.ModelCheckpoint(
'best_cnn',
save_best_only=True)

lr_schedule = LearningRateScheduler(lr_scheduler)

model = create_keras_model()

model.summary()

In [ ]:
start = time.time()
history = model.fit(train_loader, epochs=70, validation_data=val_loader, callbacks=[early_stopping,lr_schedule])
end = time.time()
print("CNN ResNet Fit Time: ", end - start,"s")